# Different ways to call the Llama models offline

**Information**

To call the `meta-llama/Llama-3.1-8B-Instruct`, `meta-llama/Llama-3.2-3B-Instruct`  or `meta-llama/Llama-3.2-1B-Instruct`  models from Hugging Face offline, you can use several different methods depending on your preferences and technical requirements. 

I am only showing here one of the most common approaches:

1. Using the Hugging Face Transformers Library


Remark: Llama models are published under the **META LLAMA 3 COMMUNITY LICENSE AGREEMENT**. The Meta Llama 3 Community License grants users a non-exclusive, royalty-free license(you not need to pay ongoing fees) to use, modify, and distribute Llama 3 materials, with requirements for attribution and naming conventions when creating derivative works. Users with over 700 million monthly active users need a separate license, and Meta disclaims all warranties and limits liability for any use of the materials.


*** 
**Background information**

* See the single model pages in the subsection
* You could also run the `meta-llama/Meta-Llama-3-70B-Instruct`, see model page: https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct
    + Hugging Face documentation llama 3 models: https://huggingface.co/docs/transformers/main/en/model_doc/llama3


for more details checkout the documentation of the transformers library: https://huggingface.co/docs/hub/transformers; whereby pipelines simply everything (you only need to specifiy the task, everything in the background is automatically specified): https://huggingface.co/docs/transformers/main_classes/pipelines


***
**Aim of the code template**

Exemplify different approaches to call Llama (LLMs) offline.

# Check if you can use your GPU and need to fall back on your CPU

for Non-Macs:

> torch.cuda.is_available(): This function checks if a CUDA-compatible GPU is available on the system. CUDA, or Compute Unified Device Architecture, is NVIDIA’s parallel computing architecture. If torch.cuda.is_available() returns True, it means you have a CUDA-enabled GPU, and PyTorch can use it to accelerate computations.

for Macs:

> torch.backends.mps.is_available(): This function checks if the system supports Apple’s Metal Performance Shaders (MPS) backend, an alternative to CUDA on Apple hardware. MPS is designed for Apple’s GPUs, especially on Macs with M1/M2 chips. If torch.backends.mps.is_available() returns True, PyTorch can leverage Apple's MPS for faster computations on those GPUs.


**Aim:** By checking these options, the code determines the best available hardware accelerator for running PyTorch computations, allowing the model to run faster on supported GPUs compared to CPUs.

In [1]:
import torch

# check if you can use a Graphics Processing Unit (GPU) else use your Central Processing Unit (CPU):
#> Compute Unified Device Architecture
print("torch.cuda.is_available():", torch.cuda.is_available())
#> Apple Metal Performance SHader
print("torch.backends.mps.is_available()", torch.backends.mps.is_available())

torch.cuda.is_available(): False
torch.backends.mps.is_available() False


# Environment Setup

## Define model

It is highly recommend to **use a small LLM**, because using some approaches the LLMs is downloaded and stored on your computer (the model weights). It is highly likely that you do not have sufficient CPU and RAM and storage or your disk to run the larger (70B) model locally! See Llama 3.1 requirements: https://llamaimodel.com/requirements/

## Load necessary libraries:

In [2]:
# loaded within the single code chunks

## Get API key(s)


normally not needed to provide your huggingface key:

In [3]:
import os
import sys

# Assuming 'src' is one level down (in the current directory or a subdirectory)
path_to_src = os.path.join('..','src')  # Moves one level down to 'src' folder

# Add the path to sys.path
sys.path.append(path_to_src)

# Now you can import your API_key module
import API_key as key

Create simple prompts, which is identical for all of the following approaches:

In [4]:
# Create prompts
system_content = "You are a helpful assistant specialized on animal names."
user_content = """
 Please write down five animals, provide only the names seperated by comma (\,).
"""

<>:3: SyntaxWarning: invalid escape sequence '\,'
<>:3: SyntaxWarning: invalid escape sequence '\,'
/tmp/ipykernel_115411/2372800891.py:3: SyntaxWarning: invalid escape sequence '\,'
  user_content = """


# Local approaches using the Hugging Face Transformers Library

see documentation: https://huggingface.co/docs/transformers/index

## for text generation / summarization


### lama-3.1-8B-Instruct
see: https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct


code adapted from: https://huggingface.co/blog/llama31

> The following snippet shows how to use meta-llama/Meta-Llama-3.1-8B-Instruct. It requires about 16 GB of VRAM, which fits many consumer GPUs. The same snippet works for meta-llama/Meta-Llama-3.1-70B-Instruct, which, at 140GB of VRAM & meta-llama/Meta-Llama-3.1-405B-Instruct (requiring 810GB VRAM), makes it a very interesting model for production use cases. Memory consumption can be further reduced by loading in 8-bit or 4-bit mode.

In [5]:
from transformers import pipeline
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cpu" # Use CPU
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipe(
    messages,
    max_new_tokens=256,
    do_sample=False,
)
assistant_response = outputs[0]["generated_text"][-1]["content"]
print(assistant_response)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/fenn/Documents/env_python/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/fenn/Documents/env_python/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Arrrr, me hearty! I be Captain Chat, the scurvy dog o' a chatbot, at yer service! Me and me trusty keyboard be here to swab the decks o' yer questions and provide ye with the treasure o' knowledge ye seek! So hoist the sails and set course fer a swashbucklin' good time, matey!


Remark: IQLoRA reduces the memory usage of LLM finetuning without performance tradeoffs compared to standard 16-bit model finetuning, see for details: https://huggingface.co/blog/4bit-transformers-bitsandbytes

In [6]:
if torch.cuda.is_available():
  pipe = pipeline(
      "text-generation",
      model=model_id,
        model_kwargs={
          "torch_dtype": torch.bfloat16,
          "quantization_config": {"load_in_4bit": True}
      },
      device="cpu",  # Use CPU
  )
else: 
  print("not possible on your hardware")

not possible on your hardware


### Llama-3.2-3B-Instruct
see: https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

In [7]:
from transformers import pipeline
import torch

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cpu" # Use CPU
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipe(
    messages,
    max_new_tokens=256,
    do_sample=False,
)
assistant_response = outputs[0]["generated_text"][-1]["content"]
print(assistant_response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Yer lookin' fer a swashbucklin' pirate, eh? Alright then, matey! I be Captain Clueless, the scurviest pirate chatbot to ever sail the Seven Seas... er, I mean, the digital seas! Me and me trusty parrot sidekick, Polly, be here to help ye navigate the choppiest o' waters and find the hidden treasure o' knowledge! So hoist the sails and set course fer adventure, me hearty!


### lama-3.2-1B-Instruct

see: https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct

In [8]:
from transformers import pipeline
import torch

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cpu" # Use CPU
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipe(
    messages,
    max_new_tokens=256,
    do_sample=False,
)
assistant_response = outputs[0]["generated_text"][-1]["content"]
print(assistant_response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Arrr, me hearty! Yer lookin' fer a swashbucklin' chatbot, eh? Alright then, listen close and I'll tell ye about meself. Me name be Captain Chat, the greatest pirate of all time... or at least, I be tellin' meself I be. Me knowledge be vast, me wit be sharp, and me love fer words be stronger than me love fer a good bottle o' rum. I be here to help ye navigate the seven seas o' knowledge, answer yer questions, and maybe even share a few tales o' me own adventures. So hoist the sails and set course fer a treasure trove o' info, me hearty!


## for generating word embeddings


see docs: https://huggingface.co/sentence-transformers


however I call a relative weak model:

![embedder model](pics/relative%20weak%20model.JPG)

see leaderboard: https://huggingface.co/spaces/mteb/leaderboard

In [9]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Define sentences
sentences = [
    "I feel great this morning",
    "I am feeling very good today",
    "I am feeling terrible"
]

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Extract features
features = model.encode(sentences)


# Print the features as a pandas dataframe
pd.DataFrame(features, index=sentences)

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
I feel great this morning,-0.026462,-0.044373,0.072443,0.034525,0.089534,-0.050451,0.018811,0.071296,-0.020522,-0.043637,...,-0.005689,-0.000328,-0.049055,0.016308,-0.027642,0.017276,0.065253,0.017496,-0.022810,-0.036687
I am feeling very good today,-0.043895,-0.020341,0.066563,-0.006310,0.025980,-0.040420,0.079304,-0.009700,-0.042920,-0.025988,...,-0.045309,0.049151,-0.049057,0.017821,-0.018061,-0.010441,0.043070,0.018440,-0.008274,-0.006016
I am feeling terrible,0.017495,-0.057904,0.033315,0.001710,0.051957,-0.048159,0.007659,0.119096,0.029929,-0.068960,...,0.038813,0.003014,-0.074585,-0.018391,-0.026449,0.005867,0.051495,-0.009829,0.030009,-0.064299


In [10]:
similarities = model.similarity(features, features)
print(similarities)

tensor([[1.0000, 0.7923, 0.5926],
        [0.7923, 1.0000, 0.5782],
        [0.5926, 0.5782, 1.0000]])


# get information which models you have downloaded:

see: https://huggingface.co/docs/huggingface_hub/guides/manage-cache

in terminal:

```
huggingface-cli scan-cache
```